In [1]:
print("starting model validation")

import os
import numpy as np
import pandas as pd
import sys
from sklearn.metrics import pairwise_distances
import math
import dill as pickle
import matplotlib.pyplot as plt
import seaborn as sns
import allensdk.core.json_utilities as ju

workingdirectory = os.popen("git rev-parse --show-toplevel").read()[:-1]
sys.path.append(workingdirectory)
os.chdir(workingdirectory)

from mcmodels.core import VoxelModelCache
from mcmodels.core.connectivity_data import get_connectivity_data
from mcmodels.core.utils import (
    get_indices_2ormore,
    get_eval_indices,
)
from mcmodels.models.expectedloss.crossvalidation import get_loss_surface_cv_spline
from mcmodels.models.expectedloss.crossvalidation import get_embedding_cv
from mcmodels.models.expectedloss.plotting import plot_loss_surface, plot_loss_scatter
from mcmodels.models.voxel.crossvalidation import CrossvalNW
from mcmodels.models.plotting import plot_loss

# load metadata
TOP_DIR = workingdirectory
INPUT_JSON = workingdirectory + "/data/meta/input_011520.json"
EXPERIMENTS_EXCLUDE_JSON = workingdirectory + "/data/meta/experiments_exclude.json"
COARSE_STRUCTURE_SET_ID = 2
DEFAULT_STRUCTURE_SET_IDS = tuple([COARSE_STRUCTURE_SET_ID])
FOLDER = workingdirectory + "/data/raw/"

input_data = ju.read(INPUT_JSON)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)
manifest_file = input_data.get("manifest_file")
manifest_file = os.path.join(TOP_DIR, manifest_file)
cache = VoxelModelCache(manifest_file=manifest_file)
st = cache.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}
major_structures = np.load(workingdirectory + "/data/meta/major_structures.npy")
major_structure_ids = np.load(workingdirectory + "/data/meta/major_structure_ids.npy")
data_info = pd.read_excel(
    workingdirectory + "/data/meta/Whole Brain Cre Image Series_curation only.xlsx",
    "all datasets curated_070919pull",
)
data_info.set_index("id", inplace=True)
with open("data/meta/leafs.pickle", "rb") as handle:
    leafs = pickle.load(handle)
ontological_order_leaves = np.load(
    workingdirectory + "/data/meta/ontological_order_leaves_v3.npy"
)
ontological_order = np.load("data/meta/ontological_order_v3.npy")
default_structures = st.get_structures_by_set_id(DEFAULT_STRUCTURE_SET_IDS)
default_structure_ids = [st["id"] for st in default_structures if st["id"] != 934]
reorder = np.asarray([4, 7, 2, 1, 10, 9, 11, 3, 5, 8, 6, 0], dtype=int)

connectivity_data = get_connectivity_data(
    cache,
    major_structure_ids,
    experiments_exclude,
    remove_injection=False,
    structure_set_id=167587189,
    folder=FOLDER,
)
connectivity_data.ai_map = ai_map
connectivity_data.leafs = leafs
connectivity_data.get_injection_hemisphere_ids()
connectivity_data.align()
connectivity_data.get_centroids()
connectivity_data.get_data_matrices(default_structure_ids)

starting model validation


/Users/samsonkoelle/miniconda3/envs/mcm_class/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.decomposition.cdnmf_fast module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.decomposition. Anything that cannot be imported from sklearn.decomposition is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/samsonkoelle/mouse_connectivity_models/mcmodels/utils.py:27: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  mpl.rcParams["text.latex.preamble"] = [r"\usepackage{amsmath}"]
/Users/samsonkoelle/miniconda3/envs/mcm_class/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in 

annotation/ccf_2017
512
703
1089
1097
315
313
354
698
771
803
477
549


In [ ]:

connectivity_data.get_crelines(data_info)
connectivity_data.get_summarystructures(data_info)

In [14]:
len(experiments_exclude)

63

In [7]:

sid0 = list(connectivity_data.structure_datas.keys())[0]
eid0 = list(connectivity_data.structure_datas[sid0].experiment_datas.keys())[0]
contra_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(
    structure_ids=ontological_order_leaves, hemisphere_id=1
)
ipsi_targetkey = connectivity_data.structure_datas[sid0].projection_mask.get_key(
    structure_ids=ontological_order_leaves, hemisphere_id=2
)
# why is this ontological_order and not ontological_order_leavs
connectivity_data.get_regionalized_normalized_data(
    ontological_order, ipsi_targetkey, contra_targetkey
)
above_thresh = {}
thresh = 1.
for m in range(12):
    sid = major_structure_ids[m]
    above_thresh[sid] = np.where(
        connectivity_data.structure_datas[sid].reg_inj.sum(axis=1) > thresh
    )[0]

In [9]:
above_thresh[1089].shape

(119,)

In [10]:
connectivity_data.structure_datas[1089].reg_inj.shape

(122, 15)

In [13]:
connectivity_data.structure_datas[1089].reg_inj.sum(axis = 1)

array([ 92.22285482,  86.94704444,  15.74930503,  31.87939723,
        38.808499  ,  18.62808836,   2.82715991,  97.73040009,
        76.96314173,  43.1752588 ,  52.08771899,  46.28863907,
       134.58049557,  13.28672998, 130.87423988,   9.16048717,
       163.84784195,  84.23641097,  35.32037612,  20.31832889,
        38.50305367,  16.45821056,   7.21147324,  10.27625108,
        48.91752228,  24.77646255,  52.86222553, 184.86096354,
         0.45270361,   3.87058234,  55.64315703,  52.54935786,
       385.60094273,   2.09677505,   5.94380331, 122.78809193,
       298.60370907, 105.1412113 , 103.76855849,  43.98730092,
        31.55429977, 140.22614589,  34.37143677,   5.27896428,
        19.02999551, 274.74255389, 140.42110492,   7.02473011,
        44.27008169, 151.80364642,  95.4914603 , 252.07355465,
       198.92646762,   9.24949455,   3.13325644, 148.41680689,
         8.15581935, 158.47872971,  70.78661102,  68.36185992,
        42.94553366,   6.10664463,  12.83690054,   5.74

In [12]:
ipsi_targetkey.shape

(448962,)

In [1]:
connectivity_data.structure_datas[1089].injections.sum(axis = 1)

NameError: name 'connectivity_data' is not defined

In [3]:
connectivity_data.structure_data[1089].injections

NameError: name 'connectivity_data' is not defined